In [26]:
import os

In [2]:
%pwd

'c:\\Users\\sr\\OneDrive\\Documents\\Work\\Agni\\Development\\Python\\TGSRTC_Productivity\\research'

In [3]:
os.chdir("../")

In [27]:
%pwd

'c:\\Users\\sr\\OneDrive\\Documents\\Work\\Agni\\Development\\Python\\TGSRTC_Productivity'

In [28]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    penalty: str #added for LR
    solver: str  #added for LR
    C: float     #added for LR
    #alpha: float
    #l1_ratio: float
    target_column: str

In [29]:
from TGSRTC_Productivity.constants import *
from TGSRTC_Productivity.utils.common import read_yaml, create_directories

In [30]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.LogisticRegression
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            penalty = params.penalty,
            solver = params.solver,
            C = params.C,
            #l1_ratio = params.l1_ratio,
            #alpha = params.alpha,
            #l1_ratio = params.l1_ratio,
            target_column = schema.name
            
        )

        return model_trainer_config

In [25]:
import pandas as pd
import os
from TGSRTC_Productivity import logger
#from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression
#import joblib
import pickle

In [35]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        #lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr = LogisticRegression(penalty=self.config.penalty, solver=self.config.solver, C=self.config.C)
        lr.fit(train_x, train_y)

        with open(os.path.join(self.config.root_dir, self.config.model_name), 'wb') as file:
            pickle.dump(lr, file)
        #joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [36]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-09-24 16:48:18,529: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-24 16:48:18,532: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-24 16:48:18,538: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-09-24 16:48:18,540: INFO: common: created directory at: artifacts]
[2024-09-24 16:48:18,542: INFO: common: created directory at: artifacts/model_trainer]


c:\Users\sr\anaconda3\envs\tgsrtc_prod\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\sr\anaconda3\envs\tgsrtc_prod\Lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
